In [1]:
import h5py
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"    
import tensorflow as tf
import sys
sys.path.append("..")
from Vocabulary import *
import math
import numpy as np



    

In [2]:
import numpy as np
import tensorflow as tf
path = r'..\\embeddings\\'
id_dict = {}
word_dict = {}

matrix = []
dimensions = 500
with open(path + "c_base2021_500d_150eV2", 'r' , encoding="utf-8")  as f:
    lines = f.readlines()
    vocab_size = len(lines)
    
    matrix = np.zeros((vocab_size,dimensions),dtype=float)
    for line in lines:
        values = line.split()
        word = values[0].strip()
        id = len(id_dict)
        id_dict[word]=id
        word_dict[id] = word
        vector = np.asarray(values[1:], "double")
        matrix[id_dict[word],:] = vector
print(len(id_dict))

257757


In [3]:
matrix_normalized = tf.nn.l2_normalize(matrix,axis = 1)
print(matrix_normalized)

tf.Tensor(
[[ 0.02847184 -0.05807904  0.01390211 ...  0.04456825 -0.05086107
   0.02113875]
 [ 0.03453062 -0.00876408 -0.019573   ... -0.00043157 -0.01531481
  -0.01608405]
 [-0.06455951  0.09349027  0.04363554 ...  0.03016072  0.05719361
   0.00498884]
 ...
 [ 0.07814373  0.00263115  0.02730266 ... -0.02809441 -0.01062798
   0.04439773]
 [-0.00820592 -0.05721911  0.01370169 ...  0.02852521 -0.04349924
   0.0636961 ]
 [-0.03498456  0.0386941   0.00280489 ... -0.01878905  0.03275915
  -0.00321864]], shape=(257757, 500), dtype=float64)


In [5]:
with open(r'.\datasets\wordsim353_sim_rel\wordsim_similarity_goldstandard.txt') as file:

    losses = []
    scores = []

    lines = file.readlines()
    sum_diff = 0
    for line in lines:
        if(len(line) <= 1):
            pass

        print("line" + line)
        word1, word2, score10 = line.split()
        score10 = float(score10)
        try:
            print(word1,word2)
            id1 = id_dict[word1]
            id2 = id_dict[word2]
            print(id1)
            print(id2)
            vector1 = matrix_normalized[id1,:]
            vector2 = matrix_normalized[id2,:]
            #print(vector1)
            #print(vector2)
            loss = np.dot(vector1,vector2)
        except: 
            loss = 0
        print(str(loss),str(score10/10.0))
        sum_diff += abs((score10/10.0) - loss)
        losses.append(loss)
        scores.append(score10/10.0)
    print('Average Distance between prediction and hand assigned is ' + str(sum_diff / len(lines)))
    

linetiger	cat	7.35

tiger cat
136112
84169
0.3635804923752262 0.735
linetiger	tiger	10.00

tiger tiger
136112
136112
0.9999999999999997 1.0
lineplane	car	5.77

plane car
12845
236945
0.32044538184683663 0.577
linetrain	car	6.31

train car
133586
236945
0.49978298681792205 0.631
linetelevision	radio	6.77

television radio
106237
196501
0.5972361984657113 0.6769999999999999
linemedia	radio	7.42

media radio
168603
196501
0.42893479677340174 0.742
linebread	butter	6.19

bread butter
125147
153605
0.5511916478020532 0.619
linecucumber	potato	5.92

cucumber potato
220483
9466
0.3686812503598021 0.592
linedoctor	nurse	7.00

doctor nurse
2756
54802
0.41657522716698897 0.7
lineprofessor	doctor	6.62

professor doctor
87565
2756
0.4047718562237175 0.662
linestudent	professor	6.81

student professor
139155
87565
0.3449256195614272 0.6809999999999999
linesmart	stupid	5.81

smart stupid
122149
85430
0.31087731497162596 0.581
linewood	forest	7.73

wood forest
146535
188765
0.38948658150629295 0.773


0.10574936023334577 0.20800000000000002
linecemetery	woodland	2.08

cemetery woodland
168526
239557
0.3255826107126373 0.20800000000000002
linepossibility	girl	1.94

possibility girl
191168
138800
0.1993095492490434 0.194
linecup	substance	1.92

cup substance
109107
183110
0.05204380337712667 0.192
lineforest	graveyard	1.85

forest graveyard
188765
78652
0.19106853673306884 0.185
linestock	egg	1.81

stock egg
7506
165774
0.14804457285643233 0.181
linemonth	hotel	1.81

month hotel
115403
186751
0.2521584918148537 0.181
lineenergy	secretary	1.81

energy secretary
95186
80572
0.2923137886456177 0.181
lineprecedent	group	1.77

precedent group
30310
250854
0.10094157813760489 0.177
lineproduction	hike	1.75

production hike
90557
174328
0.035922324848610424 0.175
linestock	phone	1.62

stock phone
7506
116865
0.16045933696894094 0.162
lineholy	sex	1.62

holy sex
6194
231369
0.14469730051083116 0.162
linestock	CD	1.31

stock CD
0 0.131
linedrink	ear	1.31

drink ear
179293
217250
0.141881073581

In [7]:
from scipy import stats
stats.spearmanr(losses, scores)

SpearmanrResult(correlation=0.604363594183674, pvalue=1.3367969815127711e-21)